In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from pyspark.sql.functions import desc
from pyspark.sql.functions import max
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import Bucketizer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

### Importing the data to dataframe df from the table created

In [3]:
df = spark.sql("select * from project_data_csv")

### How many null values in Each column

In [5]:
# reference: https://stackoverflow.com/questions/44627386/how-to-find-count-of-null-and-nan-values-for-each-column-in-a-pyspark-dataframe
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----+-------+------------+-----------------------+--------------------+-----+-----+-------------------------+-------+----------------+--------+-----------------+----------------+-----------+----------------+-----------------+----------+-------------------------+------------------------------+----------+---------+
 ID|BOROUGH|NEIGHBORHOOD|BUILDING_CLASS_CATEGORY|TAX_CLASS_AT_PRESENT|BLOCK| LOT|BUILDING_CLASS_AT_PRESENT|ADDRESS|APARTMENT_NUMBER|ZIP_CODE|RESIDENTIAL_UNITS|COMMERCIAL_UNITS|TOTAL_UNITS|LAND_SQUARE_FEET|GROSS_SQUARE_FEET|YEAR_BUILT|TAX_CLASS_AT_TIME_OF_SALE|BUILDING_CLASS_AT_TIME_OF_SALE|SALE_PRICE|SALE_DATE|
+-----+-------+------------+-----------------------+--------------------+-----+-----+-------------------------+-------+----------------+--------+-----------------+----------------+-----------+----------------+-----------------+----------+-------------------------+------------------------------+----------+---------+
36305| 36305| 36305| 36305| 36305|36305|36305| 36305| 36305| 36305| 36305| 36305| 36305| 36305| 36305| 36305| 36305| 36305| 36305| 36305| 36305|
+-----+-------+------------+-----------------------+--------------------+-----+-----+-------------------------+-------+----------------+--------+-----------------+----------------+-----------+----------------+-----------------+----------+-------------------------+------------------------------+----------+---------+

##### I have same number of null values in every column as you can see, I had '-' in my integer columns in the csv file when I changed the datatype from string to integer in databricks while importing the data to table, for columns with the blanks were replaced with null values in the entire row.

### Calculating the number of records before  dropping the nulls

In [8]:
df.count()

Out[4]: 84548

#### Dropping the Null values from the df

In [10]:
df=df.dropna(how="any")

#### Calculating the number of records after dropping the nulls

In [12]:
df.count()

Out[6]: 48243

### Displaying sample records of the data

In [14]:
df1=spark.sql("select * from project_data_csv order by sale_price desc limit 10 ")
display(df1)

ID,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
2563,1,FINANCIAL,21 OFFICE BUILDINGS,4,40,3,O4,60 WALL STREET,,10005,0,1,1,53632,1617206,1987,4,O4,1040000000,1/24/2017 0:00
2561,1,FINANCIAL,21 OFFICE BUILDINGS,4,29,1,O4,85 BROAD STREET,,10004,0,1,1,42762,993569,1983,4,O4,652000000,5/24/2017 0:00
6336,1,KIPS BAY,08 RENTALS - ELEVATOR APARTMENTS,2,934,1,D6,460-520 2ND AVENUE,,10016,894,8,902,141836,829024,1975,2,D6,620000000,12/8/2016 0:00
2054,1,FASHION,21 OFFICE BUILDINGS,4,833,11,O4,1250 BROADWAY,,10001,0,55,55,30750,645977,1969,4,O4,565000000,11/1/2016 0:00
10038,3,DOWNTOWN-FULTON FERRY,29 COMMERCIAL GARAGES,4,54,1,G7,85 JAY STREET,,11201,0,0,0,134988,0,0,4,G7,345000000,12/20/2016 0:00
6321,1,JAVITS CENTER,21 OFFICE BUILDINGS,4,732,36,O3,433-447 NINTH AVENUE,,10001,0,2,2,39779,350000,1962,4,O3,330000000,12/6/2016 0:00
9598,1,MIDTOWN WEST,21 OFFICE BUILDINGS,4,1026,21,O5,1706-1720 BROADWAY,,10019,0,2,2,8848,52150,1918,4,O5,268124175,1/5/2017 0:00
17110,4,LONG ISLAND CITY,27 FACTORIES,4,281,1,F9,31-00 47TH AVENUE,,11101,0,51,51,120000,568000,1922,4,F9,257500000,12/22/2016 0:00
2094,1,FINANCIAL,08 RENTALS - ELEVATOR APARTMENTS,2,27,9,D5,63 WALL STREET,,10005,476,6,482,17623,400531,1929,2,D5,239114603,9/30/2016 0:00
2071,1,FASHION,25 LUXURY HOTELS,4,806,76,H1,371 7 AVENUE,,10001,529,9,538,19011,473391,1929,4,H1,212500000,12/20/2016 0:00


### Count of different types of Neighbourhood

In [16]:
df.select('NEIGHBORHOOD').distinct().count()

Out[8]: 251

### Count of different types of BUILDING_CLASS_CATEGORY

In [18]:
df.select('BUILDING_CLASS_CATEGORY').distinct().count()

Out[9]: 47

### Count of different types of BUILDING_CLASS_AT_TIME_OF_SALE

In [20]:
df.select('BUILDING_CLASS_AT_TIME_OF_SALE').distinct().count()

Out[10]: 160

### We will create a temporary view for running spark sql queries to further analyze the data.

In [22]:
df.createOrReplaceTempView('ny_property_sale')

### Number of Properties in each Borough for Sale

In [24]:
%sql
select Boroughname,count(*) as number_of_properties from
(select case
when BOROUGH == 1 then 'Manhattan'
when BOROUGH == 2 then 'Bronx'
when BOROUGH == 3 then 'Brooklyn'
when BOROUGH == 4 then 'Queens'
when BOROUGH == 5 then 'Staten Island'
end as Boroughname
from ny_property_sale) group by Boroughname

Boroughname,number_of_properties
Queens,11078
Brooklyn,24047
Staten Island,5064
Manhattan,1005
Bronx,7049


### Number of Properties in each neighbourhood

In [26]:
display(spark.sql("select NEIGHBORHOOD,count(ID) as number_of_properties from ny_property_sale group by NEIGHBORHOOD order by number_of_properties desc"))

NEIGHBORHOOD,number_of_properties
BEDFORD STUYVESANT,1436
BOROUGH PARK,1245
SHEEPSHEAD BAY,1013
EAST NEW YORK,982
FLATBUSH-EAST,846
BAY RIDGE,832
FLUSHING-NORTH,798
CROWN HEIGHTS,793
CANARSIE,783
BUSHWICK,769


### Number of Properties in each BUILDING_CLASS_CATEGORY

In [28]:
display(spark.sql("select BUILDING_CLASS_CATEGORY,count(ID) as number_of_properties from ny_property_sale group by BUILDING_CLASS_CATEGORY order by number_of_properties desc"))

BUILDING_CLASS_CATEGORY,number_of_properties
01 ONE FAMILY DWELLINGS,14394
02 TWO FAMILY DWELLINGS,13043
03 THREE FAMILY DWELLINGS,3663
13 CONDOS - ELEVATOR APARTMENTS,3257
10 COOPS - ELEVATOR APARTMENTS,3178
07 RENTALS - WALKUP APARTMENTS,2784
15 CONDOS - 2-10 UNIT RESIDENTIAL,956
44 CONDO PARKING,783
04 TAX CLASS 1 CONDOS,739
22 STORE BUILDINGS,723


### Number of Properties in each BUILDING_CLASS_AT_TIME_OF_SALE

In [30]:
display(spark.sql("select BUILDING_CLASS_AT_TIME_OF_SALE,count(ID) as number_of_properties from ny_property_sale group by BUILDING_CLASS_AT_TIME_OF_SALE order by number_of_properties desc"))

BUILDING_CLASS_AT_TIME_OF_SALE,number_of_properties
A1,5247
A5,4600
B1,4059
B2,3816
C0,3663
R4,3257
D4,3175
B3,3116
A2,2098
B9,1369


### Checking the expensive building of the sales and it's details

In [32]:
spark.sql("select sale_price, borough, neighborhood from project_data_csv order by sale_price desc").show()

+----------+-------+--------------------+
sale_price|borough| neighborhood|
+----------+-------+--------------------+
1040000000| 1| FINANCIAL|
 652000000| 1| FINANCIAL|
 620000000| 1| KIPS BAY|
 565000000| 1| FASHION|
 345000000| 3|DOWNTOWN-FULTON F...|
 330000000| 1| JAVITS CENTER|
 268124175| 1| MIDTOWN WEST|
 257500000| 4| LONG ISLAND CITY|
 250867763| 1| MIDTOWN CBD|
 239114603| 1| FINANCIAL|
 217500000| 1| MIDTOWN EAST|
 212500000| 1| FASHION|
 205961801| 1|GREENWICH VILLAGE...|
 203345000| 1| FINANCIAL|
 182391612| 1| FINANCIAL|
 174000000| 1| KIPS BAY|
 173439705| 4| LONG ISLAND CITY|
 172500000| 1|UPPER WEST SIDE (...|
 160000000| 3| WILLIAMSBURG-NORTH|
 155000000| 1| MIDTOWN CBD|
+----------+-------+--------------------+
only showing top 20 rows

### Number of properties in each Neighbourhood grouped by Borough

In [34]:
A2=df.select("NEIGHBORHOOD","BOROUGH").groupby("NEIGHBORHOOD","BOROUGH").count().sort(desc("count")).show(1000,truncate = False)

+-------------------------+-------+-----+
NEIGHBORHOOD |BOROUGH|count|
+-------------------------+-------+-----+
BEDFORD STUYVESANT |3 |1436 |
BOROUGH PARK |3 |1245 |
SHEEPSHEAD BAY |3 |1013 |
EAST NEW YORK |3 |982 |
FLATBUSH-EAST |3 |846 |
BAY RIDGE |3 |832 |
FLUSHING-NORTH |4 |798 |
CROWN HEIGHTS |3 |793 |
CANARSIE |3 |783 |
BUSHWICK |3 |769 |
PARK SLOPE |3 |706 |
GRAVESEND |3 |705 |
RIVERDALE |2 |647 |
SUNSET PARK |3 |616 |
BENSONHURST |3 |576 |
OCEAN PARKWAY-NORTH |3 |571 |
FLATBUSH-CENTRAL |3 |538 |
BRIGHTON BEACH |3 |510 |
SOUNDVIEW |2 |499 |
GREAT KILLS |5 |485 |
MIDWOOD |3 |468 |
SO. JAMAICA-BAISLEY PARK |4 |467 |
GREENPOINT |3 |463 |
RICHMOND HILL |4 |461 |
PARKCHESTER |2 |455 |
WILLIAMSBURG-EAST |3 |455 |
WILLIAMSBRIDGE |2 |440 |
BAYSIDE |4 |437 |
ST. ALBANS |4 |436 |
BAYCHESTER |2 |436 |
WILLIAMSBURG-SOUTH |3 |427 |
WILLIAMSBURG-NORTH |3 |422 |
MADISON |3 |414 |
SPRINGFIELD GARDENS |4 |409 |
BROOKLYN HEIGHTS |3 |404 |
FLATBUSH-NORTH |3 |403 |
MARINE PARK |3 |397 |
CYPRESS HILLS |3 |394 |
CLINTON HILL |3 |390 |
PROSPECT HEIGHTS |3 |387 |
SOUTH OZONE PARK |4 |382 |
QUEENS VILLAGE |4 |382 |
OCEAN HILL |3 |380 |
SOUTH JAMAICA |4 |374 |
OCEAN PARKWAY-SOUTH |3 |360 |
MORRISANIA/LONGWOOD |2 |358 |
THROGS NECK |2 |352 |
OZONE PARK |4 |341 |
BRONXDALE |2 |332 |
DYKER HEIGHTS |3 |327 |
ASTORIA |4 |325 |
FLUSHING-SOUTH |4 |312 |
HOLLIS |4 |309 |
WILLIAMSBURG-CENTRAL |3 |291 |
ROSEDALE |4 |289 |
WOODHAVEN |4 |287 |
CORONA |4 |280 |
MORRIS PARK/VAN NEST |2 |276 |
DOWNTOWN-FULTON MALL |3 |276 |
BATH BEACH |3 |272 |
DOWNTOWN-FULTON FERRY |3 |265 |
SCHUYLERVILLE/PELHAM BAY |2 |264 |
CONEY ISLAND |3 |263 |
MIDDLE VILLAGE |4 |263 |
OLD MILL BASIN |3 |256 |
BOERUM HILL |3 |248 |
MELROSE/CONCOURSE |2 |244 |
ELMHURST |4 |243 |
ELTINGVILLE |5 |243 |
COLLEGE POINT |4 |241 |
JACKSON HEIGHTS |4 |238 |
WAKEFIELD |2 |235 |
FLATLANDS |3 |234 |
LAURELTON |4 |232 |
FAR ROCKAWAY |4 |227 |
BROWNSVILLE |3 |227 |
ARDEN HEIGHTS |5 |225 |
FORT GREENE |3 |224 |
BULLS HEAD |5 |218 |
GOWANUS |3 |215 |
GLENDALE |4 |213 |
HOWARD BEACH |4 |213 |
WEST NEW BRIGHTON |5 |212 |
KINGSBRIDGE/JEROME PARK |2 |210 |
MASPETH |4 |208 |
WHITESTONE |4 |207 |
BEDFORD PARK/NORWOOD |2 |203 |
PARK SLOPE SOUTH |3 |202 |
NEW SPRINGVILLE |5 |194 |
BERGEN BEACH |3 |194 |
CARROLL GARDENS |3 |191 |
MIDLAND BEACH |5 |189 |
CASTLE HILL/UNIONPORT |2 |188 |
RIDGEWOOD |4 |186 |
WINDSOR TERRACE |3 |180 |
WESTERLEIGH |5 |175 |
EAST ELMHURST |4 |175 |
PORT RICHMOND |5 |174 |
FOREST HILLS |4 |173 |
TOTTENVILLE |5 |171 |
FLATBUSH-LEFFERTS GARDEN |3 |170 |
SOUTH BEACH |5 |168 |
MARINERS HARBOR |5 |168 |
COUNTRY CLUB |2 |168 |
KENSINGTON |3 |160 |
DOWNTOWN-METROTECH |3 |157 |
BELLEROSE |4 |154 |
EAST TREMONT |2 |145 |
PELHAM PARKWAY NORTH |2 |145 |
WESTCHESTER |2 |142 |
CAMBRIA HEIGHTS |4 |140 |
MOTT HAVEN/PORT MORRIS |2 |140 |
COBBLE HILL |3 |139 |
HARLEM-CENTRAL |1 |137 |
ARVERNE |4 |135 |
PELHAM PARKWAY SOUTH |2 |134 |
MILL BASIN |3 |133 |
KINGSBRIDGE HTS/UNIV HTS |2 |130 |
NEW BRIGHTON |5 |130 |
HIGHBRIDGE/MORRIS HEIGHTS|2 |129 |
GERRITSEN BEACH |3 |123 |
WILLOWBROOK |5 |118 |
ROSEBANK |5 |116 |
ANNADALE |5 |110 |
FLORAL PARK |4 |109 |
NEW DORP |5 |107 |
WYCKOFF HEIGHTS |3 |106 |
REGO PARK |4 |106 |
JAMAICA |4 |106 |
PRINCES BAY |5 |101 |
ROSSVILLE |5 |100 |
CASTLETON CORNERS |5 |99 |
HARLEM-EAST |1 |98 |
LITTLE NECK |4 |97 |
MANOR HEIGHTS |5 |95 |
MANHATTAN BEACH |3 |94 |
WOODROW |5 |94 |
WOODSIDE |4 |91 |
CITY ISLAND |2 |91 |
LONG ISLAND CITY |4 |90 |
MIDTOWN WEST |1 |89 |
BELMONT |2 |89 |
HUGUENOT |5 |88 |
MOUNT HOPE/MOUNT EDEN |2 |88 |
WOODLAWN |2 |85 |
COBBLE HILL-WEST |3 |84 |
DOUGLASTON |4 |84 |
NEW DORP-BEACH |5 |83 |
RED HOOK |3 |83 |
STAPLETON |5 |82 |
PELHAM GARDENS |2 |80 |
CONCORD |5 |80 |
FRESH MEADOWS |4 |79 |
FORDHAM |2 |78 |
SUNNYSIDE |4 |77 |
HUNTS POINT |2 |76 |
JAMAICA ESTATES |4 |75 |
CLOVE LAKES |5 |72 |
OAKLAND GARDENS |4 |71 |
SPRING CREEK |3 |70 |
NEW DORP-HEIGHTS |5 |70 |
BATHGATE |2 |68 |
GRANT CITY |5 |68 |
BELLE HARBOR |4 |68 |
SEAGATE |3 |67 |
CROTONA PARK |2

### Number of properties by each TAX_CLASS_AT_PRESENT

In [36]:
# Most of the sales belong to Class 1 which Includes most residential property of up to three units (such as one-,two-, and three-family homes and small stores or offices with one or two attached apartments), vacant land that is zoned for residential use, and most condominiums that are not more than three stories. 
A2=df.select("TAX_CLASS_AT_PRESENT").groupby("TAX_CLASS_AT_PRESENT").count().sort(desc("count")).show(1000,truncate = False)

+--------------------+-----+
TAX_CLASS_AT_PRESENT|count|
+--------------------+-----+
1 |31194|
2 |8009 |
4 |3486 |
2A |2098 |
2C |1232 |
2B |646 |
1A |547 |
1B |511 |
 |352 |
1C |166 |
3 |2 |
+--------------------+-----+

### Statistics of each variable

In [38]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
ID,48243,10449.002653234666,7574.482564925197,4,26702
BOROUGH,48243,3.25178782414029,0.9030869090762457,1,5
NEIGHBORHOOD,48243,None,None,AIRPORT LA GUARDIA,WYCKOFF HEIGHTS
BUILDING_CLASS_CATEGORY,48243,None,None,01 ONE FAMILY DWELLINGS,49 CONDO WAREHOUSES/FACTORY/INDUS
TAX_CLASS_AT_PRESENT,48243,1.4326673069265186,0.8576226240753219,,4
BLOCK,48243,4984.608544244761,3427.41258158423,1,16319
LOT,48243,234.32949858010488,496.36552869491544,1,9009
BUILDING_CLASS_AT_PRESENT,48243,None,None,,Z9
ADDRESS,48243,None,None,1 12TH ST EXTENSION,ZEREGA AVENUE


### Correlation Matrix

In [40]:
NEIGHBORHOOD_indexer_corr = StringIndexer(inputCol='NEIGHBORHOOD',outputCol='NEIGHBORHOOD_index_corr',handleInvalid='keep')
BUILDING_CLASS_CATEGORY_indexer_corr = StringIndexer(inputCol='BUILDING_CLASS_CATEGORY',outputCol='BUILDING_CLASS_CATEGORY_index_corr',handleInvalid='keep')


# Create a vector assembler with all input features and indexed features
assembler_corr = VectorAssembler(inputCols=['BOROUGH','NEIGHBORHOOD_index_corr','BUILDING_CLASS_CATEGORY_index_corr','RESIDENTIAL_UNITS','COMMERCIAL_UNITS','TOTAL_UNITS','LAND_SQUARE_FEET','GROSS_SQUARE_FEET','YEAR_BUILT','TAX_CLASS_AT_TIME_OF_SALE','SALE_PRICE'],
                            outputCol="features")

# create a pipeline with stages of transformation
pipe = Pipeline(stages=[NEIGHBORHOOD_indexer_corr,BUILDING_CLASS_CATEGORY_indexer_corr,assembler_corr])


# fit the pipeline to our spark dataframe df
fit_pipe_corr = pipe.fit(df)


# create a transformed data frame 
df_correlated=fit_pipe_corr.transform(df)
# Getting the correlation dense matrix
corr = Correlation.corr(df_correlated, "features").collect()[0][0]
# Coverting it to a list so that it can plotted
corrmatrix = corr.toArray().tolist() 
# defining the features data columns since if convert the corr matrix to list we will loose column headers
data_columns=['BOROUGH','NEIGHBORHOOD_index_corr','BUILDING_CLASS_CATEGORY_index_corr','RESIDENTIAL_UNITS','COMMERCIAL_UNITS','TOTAL_UNITS','LAND_SQUARE_FEET','GROSS_SQUARE_FEET','YEAR_BUILT','TAX_CLASS_AT_TIME_OF_SALE','SALE_PRICE']
# We create a dataframe with correlated values list between features and features headers
corrdf=spark.createDataFrame(corrmatrix,data_columns)

print(corrdf)

# We will plot the correlation matrix with matplot matrix show.
# reference: https://stackoverflow.com/questions/55546467/how-to-plot-correlation-heatmap-when-using-pysparkdatabricks
fig, ax = plt.subplots()
# Creating a matrix plot with correlation matrix with max value 1 and min value -1
corr_ax=ax.matshow(corrmatrix,vmax=1,vmin=-1)
# Setting title xlabel , ylabel, xticks and yticks
ax.set_title("Correlation Matrix")
ax.set_xticks(np.arange(len(data_columns)))
ax.set_yticks(np.arange(len(data_columns)))
ax.set_xticklabels(data_columns)
ax.set_yticklabels(data_columns)
plt.gca().xaxis.tick_bottom()
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
plt.tick_params(axis="both", which="both" )
fig.colorbar(corr_ax)
# Adding correlation value  using every element of corr matrix on the matrix plot 
for i in range(len(corrmatrix)):
  for j in range(len(corrmatrix[0])):
          text = ax.text(j, i, "%.2f" % corrmatrix[i][j],
                         ha="center", va="center", color="white", fontsize="x-small")
fig.tight_layout()
display(fig)

### Visualization of Data

In [42]:
from pyspark.sql.types import *
def valueToCategory1(value):

   if   value == 1: return 'Manhattan'
   elif value == 2: return 'Bronx'
   elif value == 3: return 'Brooklyn'
   elif value == 4: return 'Queens'
   elif value == 5: return 'Staten Island'
   else: return 'n/a'
  

def valueToCategory2(value1):

   if   value1 == 1: return 'ResidentialProperty-3'
   elif value1 == 2: return 'PrimarilyResidential'
   elif value1 == 3: return 'OwnedByOthers'
   elif value1 == 4: return 'Companies'
   else: return 'n/a'

udfValueToCategory1 = udf(valueToCategory1, StringType())
udfValueToCategory2 = udf(valueToCategory2, StringType())


df1 = df.withColumn("Boroughname", udfValueToCategory1("BOROUGH")).withColumn("Tax_Class_Name", udfValueToCategory2("TAX_CLASS_AT_TIME_OF_SALE"))


df1.show()

+---+-------+-------------+-----------------------+--------------------+-----+---+-------------------------+--------------------+----------------+--------+-----------------+----------------+-----------+----------------+-----------------+----------+-------------------------+------------------------------+----------+---------------+-----------+--------------------+
 ID|BOROUGH| NEIGHBORHOOD|BUILDING_CLASS_CATEGORY|TAX_CLASS_AT_PRESENT|BLOCK|LOT|BUILDING_CLASS_AT_PRESENT| ADDRESS|APARTMENT_NUMBER|ZIP_CODE|RESIDENTIAL_UNITS|COMMERCIAL_UNITS|TOTAL_UNITS|LAND_SQUARE_FEET|GROSS_SQUARE_FEET|YEAR_BUILT|TAX_CLASS_AT_TIME_OF_SALE|BUILDING_CLASS_AT_TIME_OF_SALE|SALE_PRICE| SALE_DATE|Boroughname| Tax_Class_Name|
+---+-------+-------------+-----------------------+--------------------+-----+---+-------------------------+--------------------+----------------+--------+-----------------+----------------+-----------+----------------+-----------------+----------+-------------------------+------------------------------+----------+---------------+-----------+--------------------+
 4| 1|ALPHABET CITY| 07 RENTALS - WALK...| 2A| 392| 6| C2| 153 AVENUE B| | 10009| 5| 0| 5| 1633| 6440| 1900| 2| C2| 6625000| 7/19/2017 0:00| Manhattan|PrimarilyResidential|
 7| 1|ALPHABET CITY| 07 RENTALS - WALK...| 2B| 402| 21| C4| 154 EAST 7TH STREET| | 10009| 10| 0| 10| 2272| 6794| 1913| 2| C4| 3936272| 9/23/2016 0:00| Manhattan|PrimarilyResidential|
 8| 1|ALPHABET CITY| 07 RENTALS - WALK...| 2A| 404| 55| C2|301 EAST 10TH S...| | 10009| 6| 0| 6| 2369| 4615| 1900| 2| C2| 8000000|11/17/2016 0:00| Manhattan|PrimarilyResidential|
 10| 1|ALPHABET CITY| 07 RENTALS - WALK...| 2B| 406| 32| C4| 210 AVENUE B| | 10009| 8| 0| 8| 1750| 4226| 1920| 2| C4| 3192840| 9/23/2016 0:00| Manhattan|PrimarilyResidential|
 13| 1|ALPHABET CITY| 08 RENTALS - ELEV...| 2| 387|153| D9| 629 EAST 5TH STREET| | 10009| 24| 0| 24| 4489| 18523| 1920| 2| D9| 16232000| 11/7/2016 0:00| Manhattan|PrimarilyResidential|
 15| 1|ALPHABET CITY| 08 RENTALS - ELEV...| 2B| 400| 21| D1|530-532 EAST 5TH ...| | 10009| 10| 0| 10| 3717| 12350| 2009| 2| D1| 10350000|10/17/2016 0:00| Manhattan|PrimarilyResidential|
 26| 1|ALPHABET CITY| 09 COOPS - WALKUP...| 2| 376| 14| C6| 244 EAST 7TH STREET| | 10009| 24| 0| 24| 4131| 16776| 1928| 2| C6| 11900000| 6/21/2017 0:00| Manhattan|PrimarilyResidential|
176| 1|ALPHABET CITY| 14 RENTALS - 4-10...| 2A| 391| 19| S3|624 EAST 9TH ST...| | 10009| 3| 1| 4| 1520| 3360| 1910| 2| S3| 3300000|11/15/2016 0:00| Manhattan|PrimarilyResidential|
177| 1|ALPHABET CITY| 14 RENTALS - 4-10...| 2A| 393| 4| S4| 165 AVENUE B| | 10009| 4| 1| 5| 2201| 5608| 1900| 2| S4| 7215000| 1/30/2017 0:00| Manhattan|PrimarilyResidential|
178| 1|ALPHABET CITY| 14 RENTALS - 4-10...| 2A| 394| 5| S5| 183 AVENUE B| | 10009| 5| 1| 6| 1779| 3713| 1910| 2| S5| 4750000| 4/3/2017 0:00| Manhattan|PrimarilyResidential|
199| 1|ALPHABET CITY| 22 STORE BUILDING...| 4| 390| 34| K4| 115 AVENUE C| | 10009| 0| 1| 1| 1218| 3586| 1899| 4| K4| 3700000| 9/28/2016 0:00| Manhattan| Companies|
210| 1| CHELSEA| 01 ONE FAMILY DWE...| 1| 771| 52| A9|224 WEST 22ND S...| | 10011| 1| 0| 1| 1975| 3855| 1901| 1| A9| 7425000|12/30/2016 0:00| Manhattan|ResidentialProper...|
211| 1| CHELSEA| 01 ONE FAMILY DWE...| 1| 772| 29| A4|231 WEST 22ND S...| | 10011| 1| 0| 1| 1588| 5653| 1900| 1| A4| 10| 1/30/2017 0:00| Manhattan|ResidentialProper...|
212| 1| CHELSEA| 01 ONE FAMILY DWE...| 1| 772| 29| A4|231 WEST 22ND S...| | 10011| 1| 0| 1| 1588| 5653| 1900| 1| A4| 10| 1/30/2017 0:00| Manhattan|ResidentialProper...|
213| 1| CHELSEA| 01 ONE FAMILY DWE...| 1| 772| 29| A4|231 WEST 22ND S...| | 10011| 1| 0| 1| 1588| 5653| 1900| 1| A4| 10| 1/30/2017 0:00| Manhattan|ResidentialProper...|
216| 1| CHELSEA| 02 TWO FAMILY DWE...| 1| 769| 49| S2|218 WEST 20TH S...| | 10011| 2| 1| 3| 2150| 4978| 1920| 1| S2| 7750000| 4/17/2017 0:00| Manhattan|ResidentialProper...|
218| 1| CHELSEA| 03 THREE FAMILY D...| 1| 719| 67| C0|444 WEST 22ND STREET| | 10011| 3| 0| 3| 2469| 3725| 1901| 1

In [43]:
df1.createOrReplaceTempView('NewYork_property_sale')


In [44]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [45]:
# Convert the spark dataframe to pandas dataframe for better visualization
pd_df1=df1.toPandas()
print(pd_df1.columns)


Index(['ID', 'BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
 'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'BUILDING_CLASS_AT_PRESENT',
 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
 'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET',
 'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE',
 'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_PRICE', 'SALE_DATE',
 'Boroughname', 'Tax_Class_Name'],
 dtype='object')

### We will see what are top ten Building Class with highest property count for Tax_Class = Company

In [47]:
df1_agg = spark.sql("select count(ID) as number_of_properties,BUILDING_CLASS_AT_TIME_OF_SALE, Tax_Class_Name from NewYork_property_sale  group by BUILDING_CLASS_AT_TIME_OF_SALE,Tax_Class_Name order by number_of_properties ")
df1_agg=df1_agg.filter(df1_agg.Tax_Class_Name=='Companies')
df1_agg=df1_agg.sort(desc('number_of_properties')).limit(10)
pd_df1 = df1_agg.toPandas()  
g=sns.barplot(x='BUILDING_CLASS_AT_TIME_OF_SALE',y='number_of_properties',data=pd_df1)
g.set_xticklabels(g.get_xticklabels(), rotation=40,ha="right")
g.set_title('Companies')
plt.tight_layout()
plt.show()

### We will see what are top ten Building Class with highest property count for Tax_Class = ResidentialProperty-3

In [49]:
df1_agg = spark.sql("select count(ID) as number_of_properties,BUILDING_CLASS_AT_TIME_OF_SALE, Tax_Class_Name from NewYork_property_sale  group by BUILDING_CLASS_AT_TIME_OF_SALE,Tax_Class_Name order by number_of_properties ")
df1_agg=df1_agg.filter(df1_agg.Tax_Class_Name=='ResidentialProperty-3')
df1_agg=df1_agg.sort(desc('number_of_properties')).limit(10)
pd_df1 = df1_agg.toPandas()  
g=sns.barplot(x='BUILDING_CLASS_AT_TIME_OF_SALE',y='number_of_properties',data=pd_df1)
g.set_xticklabels(g.get_xticklabels(), rotation=40,ha="right")
g.set_title('ResidentialProperty-3')
plt.tight_layout()
plt.show()

### We will see what are top ten Building Class with highest property count for Tax_Class = PrimarilyResidential

In [51]:
df1_agg = spark.sql("select count(ID) as number_of_properties,BUILDING_CLASS_AT_TIME_OF_SALE, Tax_Class_Name from NewYork_property_sale  group by BUILDING_CLASS_AT_TIME_OF_SALE,Tax_Class_Name order by number_of_properties ")
df1_agg=df1_agg.filter(df1_agg.Tax_Class_Name=='PrimarilyResidential')
df1_agg=df1_agg.sort(desc('number_of_properties')).limit(10)
pd_df1 = df1_agg.toPandas()  
g=sns.barplot(x='BUILDING_CLASS_AT_TIME_OF_SALE',y='number_of_properties',data=pd_df1)
g.set_xticklabels(g.get_xticklabels(), rotation=40,ha="right")
g.set_title('PrimarilyResidential')
plt.tight_layout()
plt.show()

### We will see what are top ten Building Class with highest property count for Boroughname = Manhattan

In [53]:
df1_agg = spark.sql("select count(id) as number_of_properties,BUILDING_CLASS_AT_TIME_OF_SALE,Boroughname from NewYork_property_sale   group by BUILDING_CLASS_AT_TIME_OF_SALE,Boroughname order by number_of_properties")
df1_agg=df1_agg.filter(df1_agg.Boroughname=='Manhattan')
df1_agg=df1_agg.sort(desc('number_of_properties')).limit(10)
pd_df1 = df1_agg.toPandas()  
g=sns.barplot(x='BUILDING_CLASS_AT_TIME_OF_SALE',y='number_of_properties',data=pd_df1)
g.set_xticklabels(g.get_xticklabels(), rotation=40,ha="right")
g.set_title('Manhattan')
plt.tight_layout()
plt.show()

### We will see what are top ten Building Class with highest property count for Boroughname = Bronx

In [55]:
df1_agg = spark.sql("select count(id) as number_of_properties,BUILDING_CLASS_AT_TIME_OF_SALE,Boroughname from NewYork_property_sale   group by BUILDING_CLASS_AT_TIME_OF_SALE,Boroughname order by number_of_properties")
df1_agg=df1_agg.filter(df1_agg.Boroughname=='Bronx')
df1_agg=df1_agg.sort(desc('number_of_properties')).limit(10)
pd_df1 = df1_agg.toPandas()  
g=sns.barplot(x='BUILDING_CLASS_AT_TIME_OF_SALE',y='number_of_properties',data=pd_df1)
g.set_xticklabels(g.get_xticklabels(), rotation=40,ha="right")
g.set_title('Bronx')
plt.tight_layout()
plt.show()

### We will see what are top ten Building Class with highest property count for Boroughname = Brooklyn

In [57]:
df1_agg = spark.sql("select count(id) as number_of_properties,BUILDING_CLASS_AT_TIME_OF_SALE,Boroughname from NewYork_property_sale   group by BUILDING_CLASS_AT_TIME_OF_SALE,Boroughname order by number_of_properties")
df1_agg=df1_agg.filter(df1_agg.Boroughname=='Brooklyn')
df1_agg=df1_agg.sort(desc('number_of_properties')).limit(10)
pd_df1 = df1_agg.toPandas()  
g=sns.barplot(x='BUILDING_CLASS_AT_TIME_OF_SALE',y='number_of_properties',data=pd_df1)
g.set_xticklabels(g.get_xticklabels(), rotation=40,ha="right")
g.set_title('Brooklyn')
plt.tight_layout()
plt.show()

### We will see what are top ten Building Class with highest property count for Boroughname = Queens

In [59]:
df1_agg = spark.sql("select count(id) as number_of_properties,BUILDING_CLASS_AT_TIME_OF_SALE,Boroughname from NewYork_property_sale   group by BUILDING_CLASS_AT_TIME_OF_SALE,Boroughname order by number_of_properties")
df1_agg=df1_agg.filter(df1_agg.Boroughname=='Queens')
df1_agg=df1_agg.sort(desc('number_of_properties')).limit(10)
pd_df1 = df1_agg.toPandas()  
g=sns.barplot(x='BUILDING_CLASS_AT_TIME_OF_SALE',y='number_of_properties',data=pd_df1)
g.set_xticklabels(g.get_xticklabels(), rotation=40,ha="right")
g.set_title('Queens')
plt.tight_layout()
plt.show()

### We will see what are top ten Building Class with highest property count for Boroughname = Staten Island

In [61]:
df1_agg = spark.sql("select count(id) as number_of_properties,BUILDING_CLASS_AT_TIME_OF_SALE,Boroughname from NewYork_property_sale   group by BUILDING_CLASS_AT_TIME_OF_SALE,Boroughname order by number_of_properties")
df1_agg=df1_agg.filter(df1_agg.Boroughname=='Staten Island')
df1_agg=df1_agg.sort(desc('number_of_properties')).limit(10)
pd_df1 = df1_agg.toPandas()  
g=sns.barplot(x='BUILDING_CLASS_AT_TIME_OF_SALE',y='number_of_properties',data=pd_df1)
g.set_xticklabels(g.get_xticklabels(), rotation=40,ha="right")
g.set_title('Staten Island')
plt.tight_layout()
plt.show()

### Distrbution of Sale Price

In [63]:
fig =plt.subplots(figsize=(8,4))
ax=sns.boxplot(pd_df1['SALE_PRICE'])
display(ax)

### Linear Plot of Gross_Sqaure Feet and Land_Square Feet on Sales Price.

In [65]:
fig1, ax =plt.subplots(figsize=(8,4))
ax=sns.scatterplot(pd_df1['GROSS_SQUARE_FEET'],pd_df1['SALE_PRICE'])
plt.tight_layout()
display(fig1)

fig, ax =plt.subplots(figsize=(8,4))
ax=sns.scatterplot(pd_df1['LAND_SQUARE_FEET'],pd_df1['SALE_PRICE'])
plt.tight_layout()
display(fig)

### Linearity Plot of Residential_Units and Commercial_Units on Sales Price.

In [67]:
fig1, ax =plt.subplots(figsize=(8,4))
ax=sns.scatterplot(pd_df1['RESIDENTIAL_UNITS'],pd_df1['SALE_PRICE'])
plt.tight_layout()
display(fig1)

fig, ax =plt.subplots(figsize=(8,4))
ax=sns.scatterplot(pd_df1['COMMERCIAL_UNITS'],pd_df1['SALE_PRICE'])
plt.tight_layout()
display(fig)

### Checking the range of Sale Price by creating buckets

In [69]:
#Referencfe : stack overflow

dffilter=df.select('sale_price')
splits = [0, 80000.0, 8500000.0,15000000,1000000000.0,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="sale_price", outputCol="bucketedFeatures")
bucketedData = bucketizer.transform(dffilter)
bucketedData.groupBy("bucketedFeatures").count().show()

+----------------+-----+
bucketedFeatures|count|
+----------------+-----+
 0.0|12031|
 1.0|35521|
 4.0| 1|
 3.0| 349|
 2.0| 341|
+----------------+-----+

### Linear Regression Including all Variables

In [71]:
df = spark.sql("select * from project_data_csv")

In [72]:
df.count()

Out[2]: 84548

In [73]:
df = df.dropna(how="any")

In [74]:
df.count()

Out[4]: 48243

### Building the Linear Regression Model

In [76]:

train_data_lr,test_data_lr=df.randomSplit([0.7,0.3],seed=12345)


neighbourhood_indexer_lr = StringIndexer(inputCol='NEIGHBORHOOD',outputCol='neighbourhood_index_lr',handleInvalid='keep')
BUILDING_CLASS_CATEGORY_Indexer_lr = StringIndexer(inputCol='BUILDING_CLASS_CATEGORY',outputCol='BUILDING_CLASS_CATEGORY_Index_lr',handleInvalid='keep')
BUILDING_CLASS_AT_TIME_OF_SALE_indexer_lr = StringIndexer(inputCol='BUILDING_CLASS_AT_TIME_OF_SALE',outputCol='BUILDING_CLASS_index_lr',handleInvalid='keep')



# Vector assembler is used to create a vector of input features

assembler_lr=VectorAssembler(inputCols=['BOROUGH','ZIP_CODE','RESIDENTIAL_UNITS','COMMERCIAL_UNITS','LAND_SQUARE_FEET','GROSS_SQUARE_FEET','YEAR_BUILT','TAX_CLASS_AT_TIME_OF_SALE',
                                     'neighbourhood_index_lr',  'BUILDING_CLASS_index_lr','BUILDING_CLASS_CATEGORY_Index_lr'],
                            outputCol="features")

# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data

pipe = Pipeline(stages=[neighbourhood_indexer_lr,BUILDING_CLASS_CATEGORY_Indexer_lr,BUILDING_CLASS_AT_TIME_OF_SALE_indexer_lr,assembler_lr])

fitted_pipe_lr=pipe.fit(train_data_lr)

train_data_lr=fitted_pipe_lr.transform(train_data_lr)

# Create an object for the Linear Regression model

lr_model = LinearRegression(labelCol='SALE_PRICE')

# Fit the model on the train data

fit_model_lr = lr_model.fit(train_data_lr.select(['features','SALE_PRICE']))

# Transform the test data using the model to predict the duration

test_data_lr=fitted_pipe_lr.transform(test_data_lr)

# Store the results in a dataframe

results_lr = fit_model_lr.transform(test_data_lr)

results_lr.select(['sale_price','prediction']).show()

+----------+------------------+
sale_price| prediction|
+----------+------------------+
 0| 554507.9139538179|
 127000| 2048470.26253072|
 3936272| 7457315.97160336|
 273796|1960461.2516669603|
 3192840| 7057468.386585165|
 290000|1953697.4866330104|
 1060000|2672493.5017875833|
 0| 2206478.380511568|
 0| 2189960.667627508|
 0| 484696.4294275944|
 540625| 2222823.99673475|
 250000|2621177.0793174743|
 18000| 2290056.27319931|
 815000|518045.69133198005|
 1150000| 605877.321842622|
 1220000| 652676.8979593185|
 740000| 959633.784046774|
 928000|1005543.0422171878|
 0| 576719.2310566967|
 10| 576719.2310566967|
+----------+------------------+
only showing top 20 rows

In [77]:
test_results_lr = fit_model_lr.evaluate(test_data_lr)

In [78]:
test_results_lr.rootMeanSquaredError

Out[12]: 9995814.302226897

In [79]:
test_results_lr.residuals.show()

+-------------------+
 residuals|
+-------------------+
 -554507.9139538179|
 -1921470.26253072|
 -3521043.97160336|
-1686665.2516669603|
 -3864628.386585165|
-1663697.4866330104|
-1612493.5017875833|
 -2206478.380511568|
 -2189960.667627508|
 -484696.4294275944|
 -1682198.99673475|
-2371177.0793174743|
 -2272056.27319931|
 296954.30866801995|
 544122.678157378|
 567323.1020406815|
-219633.78404677403|
 -77543.04221718782|
 -576719.2310566967|
 -576709.2310566967|
+-------------------+
only showing top 20 rows

In [80]:
test_results.r2

Out[15]: 0.11355501109349597

In [81]:
print("Coefficients: %s" % str(fit_model_lr.coefficients))
print("Intercept: %s" % str(fit_model_lr.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary_lr = fit_model_lr.summary
print("numIterations: %d" % trainingSummary_lr.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary_lr.objectiveHistory))
#trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary_lr.rootMeanSquaredError)
print("r2: %f" % trainingSummary_lr.r2)

Coefficients: [-435678.3070948225,-9.4222377687101,25451.582597063356,192230.17079003766,-43.469518481157984,144.7770906862842,21.72291587774203,-33714.84381595259,11783.328881653953,93624.58193067511,-123272.5983298684]
Intercept: 964962.7194534899
numIterations: 1
objectiveHistory: [0.0]
RMSE: 6891867.699489
r2: 0.256300

In [82]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd_df_lr = results_lr.toPandas()
fig, ax=plt.subplots(1,1,figsize=(10,6))
ax=sns.regplot(pd_df_lr['prediction'],pd_df_lr['SALE_PRICE'])
display(plt.show())

#display(pd_df_lr)

### Function to know the bounds for each column based on inter quartile range ( Outliers detection)

In [84]:
# Reference
#https://blog.zhaytam.com/2019/07/15/outliers-detection-in-pyspark-2-interquartile-range/


def calculate_bounds(df):
  bounds = {
    c: dict(
      zip(["q1", "q3"], df.approxQuantile(c, [0.25, 0.75], 0))
        )
    for c,d in zip(df.columns, df.dtypes) if d[1] == "int"
    }
  for c in bounds:
    iqr = bounds[c]['q3'] - bounds[c]['q1']
    bounds[c]['min'] = bounds[c]['q1'] - (iqr * 1.5)
    bounds[c]['max'] = bounds[c]['q3'] + (iqr * 1.5)

    return bounds

### Calculating Bounds using created function

In [86]:
calculate_bounds(df)

Out[29]: {'ID': {'q1': 3919.0, 'q3': 17160.0, 'min': -15942.5, 'max': 37021.5},
 'BOROUGH': {'q1': 3.0, 'q3': 4.0},
 'BLOCK': {'q1': 2207.0, 'q3': 6996.0},
 'LOT': {'q1': 21.0, 'q3': 94.0},
 'ZIP_CODE': {'q1': 10469.0, 'q3': 11237.0},
 'RESIDENTIAL_UNITS': {'q1': 1.0, 'q3': 2.0},
 'COMMERCIAL_UNITS': {'q1': 0.0, 'q3': 0.0},
 'TOTAL_UNITS': {'q1': 1.0, 'q3': 2.0},
 'LAND_SQUARE_FEET': {'q1': 1413.0, 'q3': 3071.0},
 'GROSS_SQUARE_FEET': {'q1': 828.0, 'q3': 2520.0},
 'YEAR_BUILT': {'q1': 1920.0, 'q3': 1961.0},
 'TAX_CLASS_AT_TIME_OF_SALE': {'q1': 1.0, 'q3': 2.0},
 'SALE_PRICE': {'q1': 80000.0, 'q3': 830000.0}}

In [87]:
df1=df.filter(df.SALE_PRICE.between('80000','50000000'))

In [88]:
df1.count()

Out[31]: 36117

### LinearModel 2 with filtered rows as stated above and omitted few columns as per the Correlation matrix

In [90]:
train_data_lr2,test_data_lr2=df1.randomSplit([0.8,0.2],seed=59688144)


neighbourhood_indexer_lr2 = StringIndexer(inputCol='NEIGHBORHOOD',outputCol='neighbourhood_index_lr2',handleInvalid='keep')
BUILDING_CLASS_CATEGORY_Indexer_lr2 = StringIndexer(inputCol='BUILDING_CLASS_CATEGORY',outputCol='BUILDING_CLASS_CATEGORY_Index_lr2',handleInvalid='keep')
BUILDING_CLASS_AT_TIME_OF_SALE_indexer_lr2 = StringIndexer(inputCol='BUILDING_CLASS_AT_TIME_OF_SALE',outputCol='BUILDING_CLASS_index_lr2',handleInvalid='keep')



# Vector assembler is used to create a vector of input features

assembler_lr2=VectorAssembler(inputCols=['BOROUGH','RESIDENTIAL_UNITS','COMMERCIAL_UNITS','BUILDING_CLASS_index_lr2',
                                        'LAND_SQUARE_FEET','TAX_CLASS_AT_TIME_OF_SALE',
                                     'neighbourhood_index_lr2',],
                            outputCol="features")

# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data

pipe_lr2 = Pipeline(stages=[neighbourhood_indexer_lr2,BUILDING_CLASS_CATEGORY_Indexer_lr2,BUILDING_CLASS_AT_TIME_OF_SALE_indexer_lr2,assembler_lr2])

fitted_pipe_lr2=pipe_lr2.fit(train_data_lr2)

train_data_lr2=fitted_pipe_lr2.transform(train_data_lr2)

# Create an object for the Linear Regression model

lr_model_lr2 = LinearRegression(labelCol='sale_price')

# Fit the model on the train data

fit_model_lr2 = lr_model_lr2.fit(train_data_lr2.select(['features','sale_price']))

# Transform the test data using the model to predict the duration

test_data_lr2=fitted_pipe_lr2.transform(test_data_lr2)

# Store the results in a dataframe

results_lr2 = fit_model_lr2.transform(test_data_lr2)

results_lr2.select(['sale_price','prediction']).show()

+----------+------------------+
sale_price| prediction|
+----------+------------------+
 6625000| 3865902.221730073|
 140000|1768394.0819083685|
 8000000|3894211.2374969153|
 494000|1919222.3939832796|
 748000| 895483.6426075029|
 240000|2458577.0709395707|
 850000| 892930.1163738745|
 1388000|1120716.3892110102|
 1350000|1847992.3615882741|
 299000| 2280898.265171203|
 689399| 2294964.299508986|
 800000|1121705.6487688322|
 4680000| 1852110.154497709|
 1315000|3199317.6603500876|
 2000000| 961613.0378802309|
 1875000|3329061.9922886137|
 1030000|1125539.0295553932|
 400000| 900400.089964122|
 938000| 1515305.795552707|
 350000| 905711.1772151797|
+----------+------------------+
only showing top 20 rows

In [91]:
test_results_lr2 = fit_model_lr2.evaluate(test_data_lr2)

In [92]:
test_results_lr2.rootMeanSquaredError

Out[46]: 1946584.7217804187

In [93]:
test_results_lr2.r2 

Out[47]: 0.2798358995880518

In [94]:
pd_df_lr2 = results_lr2.toPandas()
fig, ax=plt.subplots(figsize=(10,6))
ax=sns.regplot(pd_df_lr2['prediction'],pd_df_lr2['SALE_PRICE'])
display(plt.show())

### Decision Tree Model (Basic)

In [96]:
df = spark.sql("select * from project_data_csv")

In [97]:
df = df.dropna(how="any")

In [98]:
df.count()

Out[52]: 48243

In [99]:
from pyspark.ml.regression import DecisionTreeRegressor

train_data_dt,test_data_dt=df.randomSplit([0.7,0.3],seed = 50945803)
neighbourhood_indexer_dt = StringIndexer(inputCol='NEIGHBORHOOD',outputCol='neighbourhood_index_dt',handleInvalid='keep')
BUILDING_CLASS_CATEGORY_Indexer_dt = StringIndexer(inputCol='BUILDING_CLASS_CATEGORY',outputCol='BUILDING_CLASS_CATEGORY_Index_dt',handleInvalid='keep')
BUILDING_CLASS_AT_TIME_OF_SALE_indexer_dt = StringIndexer(inputCol='BUILDING_CLASS_AT_TIME_OF_SALE',outputCol='BUILDING_CLASS_index_dt',handleInvalid='keep')

assembler_dt=VectorAssembler(inputCols=['BOROUGH','ZIP_CODE','RESIDENTIAL_UNITS','COMMERCIAL_UNITS','LAND_SQUARE_FEET','GROSS_SQUARE_FEET','YEAR_BUILT','TAX_CLASS_AT_TIME_OF_SALE',
                                     'neighbourhood_index_dt', 'BUILDING_CLASS_index_dt','BUILDING_CLASS_CATEGORY_Index_dt'],
                            outputCol="features")


dt_model = DecisionTreeRegressor(labelCol='SALE_PRICE',maxBins=5000)
pipe_dt = Pipeline(stages=[neighbourhood_indexer_dt,BUILDING_CLASS_CATEGORY_Indexer_dt,BUILDING_CLASS_AT_TIME_OF_SALE_indexer_dt,assembler_dt,dt_model])
fit_model_dt=pipe_dt.fit(train_data_dt)
results_dt = fit_model_dt.transform(test_data_dt)
results_dt.select(['sale_price','Prediction']).show()

+----------+-----------------+
sale_price| Prediction|
+----------+-----------------+
 6625000|7893945.601398601|
 127000|659803.6299981686|
 0|659803.6299981686|
 638000|659803.6299981686|
 10350000|4909571.355844156|
 875025|659803.6299981686|
 748000|659803.6299981686|
 290000|659803.6299981686|
 1150000|659803.6299981686|
 452500|659803.6299981686|
 0|659803.6299981686|
 321065|659803.6299981686|
 689399|659803.6299981686|
 800000|659803.6299981686|
 0|659803.6299981686|
 0|659803.6299981686|
 4680000|659803.6299981686|
 400000|659803.6299981686|
 25000|659803.6299981686|
 826819|659803.6299981686|
+----------+-----------------+
only showing top 20 rows

In [100]:
from pyspark.ml.evaluation import RegressionEvaluator
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="SALE_PRICE", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(results_dt)

print (rmse)

11471880.09365281

In [101]:
evaluator1 = RegressionEvaluator(labelCol="SALE_PRICE", predictionCol="prediction", metricName="r2")
accuracy = evaluator1.evaluate(results_dt)
print(accuracy)

0.08410717765142917

### Decision Tree Model 2 filtered Sale Price and omitted columns based on Analysis

In [103]:
df1=df.filter(df.SALE_PRICE.between('80000','50000000'))

In [104]:
# Create a 70-30 train test split

train_data_dt2,test_data_dt2=df1.randomSplit([0.8,0.2],seed = 50945803)


neighbourhood_indexer_dt2 = StringIndexer(inputCol='NEIGHBORHOOD',outputCol='neighbourhood_index_dt2',handleInvalid='keep')
BUILDING_CLASS_CATEGORY_Indexer_dt2 = StringIndexer(inputCol='BUILDING_CLASS_CATEGORY',outputCol='BUILDING_CLASS_CATEGORY_Index_dt2',handleInvalid='keep')
BUILDING_CLASS_AT_TIME_OF_SALE_indexer_dt2 = StringIndexer(inputCol='BUILDING_CLASS_AT_TIME_OF_SALE',outputCol='BUILDING_CLASS_index_dt2',handleInvalid='keep')

assembler_dt2=VectorAssembler(inputCols=['BOROUGH','RESIDENTIAL_UNITS','COMMERCIAL_UNITS','LAND_SQUARE_FEET','GROSS_SQUARE_FEET','TAX_CLASS_AT_TIME_OF_SALE',
                                     'neighbourhood_index_dt2', 'BUILDING_CLASS_CATEGORY_Index_dt2'],
                            outputCol="features")

dt_model_2 = DecisionTreeRegressor(labelCol='SALE_PRICE',maxBins=15000)

# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data

pipe_dt2 = Pipeline(stages=[neighbourhood_indexer_dt2,BUILDING_CLASS_CATEGORY_Indexer_dt2,BUILDING_CLASS_AT_TIME_OF_SALE_indexer_dt2,assembler_dt2,dt_model_2])

fit_model_dt2=pipe_dt2.fit(train_data_dt2)

results_dt2 = fit_model_dt2.transform(test_data_dt2)

results_dt2.select(['sale_price','Prediction']).show()

+----------+------------------+
sale_price| Prediction|
+----------+------------------+
 6625000| 5862797.708333333|
 3936272| 8117601.529411765|
 2050000| 773382.9976538531|
 875025| 469895.2806206176|
 815000| 773382.9976538531|
 999000| 773382.9976538531|
 800000| 773382.9976538531|
 610000| 773382.9976538531|
 1700000|1003651.2281976744|
 1400000| 773382.9976538531|
 1130000| 773382.9976538531|
 1200000|1003651.2281976744|
 950000| 773382.9976538531|
 325000| 469895.2806206176|
 400000| 469895.2806206176|
 1300000|1635653.2429149798|
 317200| 469895.2806206176|
 1170000| 773382.9976538531|
 1135000| 773382.9976538531|
 335000| 469895.2806206176|
+----------+------------------+
only showing top 20 rows

In [105]:
# Select (prediction, true label) and compute test error
evaluator2 = RegressionEvaluator(labelCol="SALE_PRICE", predictionCol="prediction", metricName="rmse")
rmse = evaluator2.evaluate(results_dt2)
print (rmse)

2048062.8346220853

In [106]:
evaluator3 = RegressionEvaluator(labelCol="SALE_PRICE", predictionCol="prediction", metricName="r2")
accuracy1 = evaluator3.evaluate(results_dt2)
print(accuracy1)

0.3227192699082768

### K Means Clustering

In [108]:
data=df.select(['BOROUGH','NEIGHBORHOOD','BUILDING_CLASS_CATEGORY','TAX_CLASS_AT_PRESENT','ZIP_CODE','RESIDENTIAL_UNITS','COMMERCIAL_UNITS',
                                 'TOTAL_UNITS','LAND_SQUARE_FEET','GROSS_SQUARE_FEET','YEAR_BUILT','TAX_CLASS_AT_TIME_OF_SALE','BUILDING_CLASS_AT_TIME_OF_SALE','SALE_PRICE'])

In [109]:
data=data.dropna()

In [110]:
data.count()

Out[63]: 48243

In [111]:
from pyspark.ml.clustering import KMeans

In [112]:
BOROUGH_KM_indexer = StringIndexer(inputCol='BOROUGH',outputCol='BOROUGH_KM_index',handleInvalid='keep')
NEIGHBORHOOD_KM_indexer = StringIndexer(inputCol='NEIGHBORHOOD',outputCol='NEIGHBORHOOD_KM_index',handleInvalid='keep')
ZIP_CODE_KM_indexer = StringIndexer(inputCol='ZIP_CODE',outputCol='ZIP_CODE_KM_index',handleInvalid='keep')
SALE_PRICE_KM_indexer = StringIndexer(inputCol='SALE_PRICE',outputCol='SALE_PRICE_KM_index',handleInvalid='keep')


# Vector assembler is used to create a vector of input features

assembler_km = VectorAssembler(inputCols=['BOROUGH_KM_index','NEIGHBORHOOD_KM_index','ZIP_CODE_KM_index'],outputCol="features")



# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data.

pipe_km = Pipeline(stages=[BOROUGH_KM_indexer,NEIGHBORHOOD_KM_indexer,ZIP_CODE_KM_indexer,SALE_PRICE_KM_indexer,assembler_km])


final_data_km=pipe_km.fit(data).transform(data)


kmeans_model = KMeans(k=4)


fit_model_km = kmeans_model.fit(final_data_km)


wssse = fit_model_km.computeCost(final_data_km)
print("The within set sum of squared error of the mode is {}".format(wssse))


centers = fit_model_km.clusterCenters()


print("Cluster Centers")
index=1
for cluster in centers:
    print("Centroid {}: {}".format(index,cluster))
    index+=1
    
    
    
results_km = fit_model_km.transform(final_data_km)


results_km.select(['BOROUGH_KM_index','NEIGHBORHOOD_KM_index','ZIP_CODE_KM_index','prediction']).show()


results_km.groupby('prediction').count().sort('prediction').show()

The within set sum of squared error of the mode is 43893297.56942585
Cluster Centers
Centroid 1: [ 1.14983113 52.07583666 78.46412854]
Centroid 2: [ 0.36610018 19.35884582 21.1824999 ]
Centroid 3: [ 1.55425253 100.19480815 24.82557213]
Centroid 4: [ 2.04173137 155.24534161 100.32666925]
+----------------+---------------------+-----------------+----------+
BOROUGH_KM_index|NEIGHBORHOOD_KM_index|ZIP_CODE_KM_index|prediction|
+----------------+---------------------+-----------------+----------+
 4.0| 225.0| 160.0| 3|
 4.0| 225.0| 160.0| 3|
 4.0| 225.0| 160.0| 3|
 4.0| 225.0| 160.0| 3|
 4.0| 225.0| 160.0| 3|
 4.0| 225.0| 160.0| 3|
 4.0| 225.0| 160.0| 3|
 4.0| 225.0| 160.0| 3|
 4.0| 225.0| 160.0| 3|
 4.0| 225.0| 160.0| 3|
 4.0| 225.0| 160.0| 3|
 4.0| 182.0| 134.0| 3|
 4.0| 182.0| 134.0| 3|
 4.0| 182.0| 134.0| 3|
 4.0| 182.0| 134.0| 3|
 4.0| 182.0| 134.0| 3|
 4.0| 182.0| 134.0| 3|
 4.0| 182.0| 153.0| 3|
 4.0| 182.0| 134.0| 3|
 4.0| 182.0| 134.0| 3|
+----------------+---------------------+-----------------+----------+
only showing top 20 rows

+----------+-----+
prediction|count|
+----------+-----+
 0| 9771|
 1|24537|
 2| 8783|
 3| 5152|
+----------+-----+

### Random Forest Regression Model

In [114]:
df = spark.sql("select * from project_data_csv")

In [115]:
df = df.dropna(how="any")
df.count()

Out[67]: 48243

In [116]:
df=df.filter(df.SALE_PRICE.between('80000','50000000'))

In [117]:
train_data_rf,test_data_rf=df.randomSplit([0.7,0.3],seed=12345)


neighbourhood_indexer_rf= StringIndexer(inputCol='NEIGHBORHOOD',outputCol='neighbourhood_index_rf',handleInvalid='keep')
BUILDING_CLASS_CATEGORY_Indexer_rf = StringIndexer(inputCol='BUILDING_CLASS_CATEGORY',outputCol='BUILDING_CLASS_CATEGORY_Index_rf',handleInvalid='keep')
BUILDING_CLASS_AT_TIME_OF_SALE_indexer_rf = StringIndexer(inputCol='BUILDING_CLASS_AT_TIME_OF_SALE',outputCol='BUILDING_CLASS_index_rf',handleInvalid='keep')



# Vector assembler is used to create a vector of input features

assembler_rf=VectorAssembler(inputCols=['BOROUGH','ZIP_CODE','RESIDENTIAL_UNITS','COMMERCIAL_UNITS','LAND_SQUARE_FEET','GROSS_SQUARE_FEET','YEAR_BUILT','TAX_CLASS_AT_TIME_OF_SALE',
                                        'neighbourhood_index_rf'
                                     ],
                            outputCol="features")

# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data

pipe_rf = Pipeline(stages=[neighbourhood_indexer_rf,BUILDING_CLASS_CATEGORY_Indexer_rf,BUILDING_CLASS_AT_TIME_OF_SALE_indexer_rf,assembler_rf])

fitted_pipe_rf=pipe_rf.fit(train_data_rf)

train_data_rf=fitted_pipe_rf.transform(train_data_rf)

# Create an object for the Linear Regression model



rf_model=RandomForestRegressor(featuresCol="features",labelCol='sale_price',maxBins=300)

# Fit the model on the train data

fit_model_rf= rf_model.fit(train_data_rf.select(['features','sale_price']))

# Transform the test data using the model to predict the duration

test_data_rf=fitted_pipe_rf.transform(test_data_rf)

# Store the results in a dataframe

results_rf = fit_model_rf.transform(test_data_rf)

results_rf.select(['sale_price','prediction']).show()

+----------+------------------+
sale_price| prediction|
+----------+------------------+
 127000| 524064.4027538144|
 3936272| 7484764.403355377|
 1200000| 982980.552720877|
 8000000| 7104337.887760366|
 950000| 903776.3001253875|
 16232000| 9558689.467451524|
 530000| 545247.4018443538|
 400000| 703412.5958318653|
 750000| 836352.8958869984|
 200000| 513731.7736656094|
 1388000| 836352.8958869984|
 1150000| 836352.8958869984|
 740000| 836352.8958869984|
 928000| 836352.8958869984|
 800000| 903776.3001253875|
 4461000|5072700.3604392735|
 880000| 834012.6776096226|
 1030000| 879182.4373576676|
 700000| 903776.3001253875|
 1800000| 8193279.084103246|
+----------+------------------+
only showing top 20 rows

In [118]:
evaluate_rf = RegressionEvaluator(
    labelCol="SALE_PRICE", predictionCol="prediction", metricName="r2")
r2 = evaluate_rf.evaluate(results_rf)
print("R Squared (R2) on test data = %g" % r2)

R Squared (R2) on test data = 0.519004

In [119]:
pd_df_lr = results_rf.toPandas()
fig, ax=plt.subplots(figsize=(10,6))
ax=sns.regplot(pd_df_lr['prediction'],pd_df_lr['SALE_PRICE'])
display(plt.show())


### Conclusions
1) The correlation matrix suggests the maximum correlation for sale price is with gross square feet column which is around 0.42.

2) To predict the price more accurately, we would need more features that have more influence on the price and are critical to price of the listings.

3) The New York sales dataset analysis provided us with finding interesting relations between various features which influence the buying of a property.

4) The sales feature having the most correlation with gross square feet makes sense because the sales will depend on the area of the flat.

5) As from the models We can observe that the R2 for linear regression models is approximately 12 and 28 which seem to be considerably less which concludes to a point that the performance is not so good

6) Coming to the Random forest regression model , We can observe that the R2 value is around 51 which brings us to a conclusion that this model is the best fit for this data set

7) The accuracy for the decision tree is 32 which is considerably a good model

### Summary
• There are 5 distinct boroughs in New York city

• There is a total of 251 neighborhoods in which the properties are listed.

• There are 47 different types of “Building Class Category”

• There are 160 different types of “Building class at the time of sale”

• We can observe that among all the boroughs “Brooklyn” has the highest number of properties with a count of 24047 and is the most valued borough

• “Bedford Stuyvesant” is the neighborhood with highest number of properties among the neighborhoods with 1436 properties which confirms it to be the most popular neighborhood

• There are 14394 properties in the “Building class category” of ‘One Family Dwellings’

• There are 5247 properties in the” Building class at the time of sale ” of “A”

• For the building class at the time of sales “RG” has the highest number of properties with a count close to 600

• For the Residential Property 3 there are about 800 properties in Bedford Stuyvesant neighborhood

• Riverdale is the neighborhood with highest number of primary residential properties with about 520

• In the Manhattan borough Harlem-central is the neighborhood with highest number of properties

• In the Bronx borough Riverdale is the neighborhood with highest number of properties

• In the Brooklyn borough Bedford Stuyvesant is the neighborhood with highest number of properties

• In the Queens borough Flushing-North is the neighborhood with highest number of properties

• In the borough of Staten Island Great Kills is the neighborhood with highest number of properties

• There are more number of residential units in the range of 0-250

• There are more number of commercial units in the initial price range though with few outliers